In [1]:
import pandas as pd
import requests
import json
import re
from bs4 import BeautifulSoup
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
!ls

Data Analysis For Profit vs. Non Profit.ipynb
FOIA.docx
NYS-Nursing-Home-Staffing-Ownership-Data-2019-UPDATED-9_10_19.xlsx
Nursing Homes in Texas Proposal.docx
Nursing Homes memo.docx
Ownership (1).csv
PBJ_Daily_Nurse_Staffing_CY_2019Q1.csv
all_homes.csv
corp_type.csv
ownershipd3.csv
ownershipdata.csv
text2567_20190801_cms_reg1.xlsx
text2567_20190801_cms_reg10.xlsx
text2567_20190801_cms_reg2.xlsx
text2567_20190801_cms_reg3.xlsx
text2567_20190801_cms_reg4.xlsx
text2567_20190801_cms_reg5a.xlsx
text2567_20190801_cms_reg5b.xlsx
text2567_20190801_cms_reg6.xlsx
text2567_20190801_cms_reg7.xlsx
text2567_20190801_cms_reg8.xlsx
text2567_20190801_cms_reg9.xlsx
tx.csv


In [3]:
region1 = pd.read_excel("text2567_20190801_cms_reg1.xlsx")

In [4]:
region2 = pd.read_excel("text2567_20190801_cms_reg2.xlsx")

In [5]:
region3 = pd.read_excel("text2567_20190801_cms_reg3.xlsx")

In [6]:
region4 = pd.read_excel("text2567_20190801_cms_reg4.xlsx")

In [7]:
region5a = pd.read_excel("text2567_20190801_cms_reg5a.xlsx")

In [8]:
region5b = pd.read_excel("text2567_20190801_cms_reg5b.xlsx")

In [9]:
region6 = pd.read_excel("text2567_20190801_cms_reg6.xlsx")

In [10]:
region7 = pd.read_excel("text2567_20190801_cms_reg7.xlsx")

In [11]:
region8 = pd.read_excel("text2567_20190801_cms_reg8.xlsx")

In [12]:
region9 = pd.read_excel("text2567_20190801_cms_reg9.xlsx")

In [13]:
region10 = pd.read_excel("text2567_20190801_cms_reg10.xlsx")

In [14]:
frames = [region1, region2, region3, region4, region5a, region5b, region6, region7, region8, region9, region10]

In [15]:
df = pd.concat(frames)

In [16]:
urls = df.groupby(by='facility_id').count().reset_index().drop(['facility_name', 'address', 'city', 'state', 'zip', 'inspection_date', 'deficiency_tag', 'scope_severity', 'complaint', 'standard', 'eventid', 'inspection_text'], axis=1)

# Non-Profit vs. For-Profit Ownership

## Scraping

In [17]:
urls['URL'] = 'https://www.medicare.gov/nursinghomecompare/profile.html#profTab=0&ID=' + urls.facility_id

In [18]:
urls.iloc[0,1]

'https://www.medicare.gov/nursinghomecompare/profile.html#profTab=0&ID=015009'

In [19]:
urls['slug'] = urls.URL.str.replace('https://www.medicare.gov/nursinghomecompare/profile.html#profTab=0&ID=', "")

In [20]:
urls.dtypes

facility_id    object
URL            object
slug           object
dtype: object

In [21]:
# response = requests.get('https://www.medicare.gov/service/nhc/provider/getprovidersbyid/?ID=015436')
# example = response.json()

In [22]:
# example

In [23]:
# example[0]['measureInfo'][-2]['scr']

In [24]:
# def fetch_corp_type(slug):
#     response = requests.get("https://www.medicare.gov/service/nhc/provider/getprovidersbyid/?ID=" + str(slug))
#     example = response.json()
#     try:
#         return example[0]["measureInfo"][-2]["scr"]
#     except:
#         return "Poorly formatted :("

In [25]:
# urls['corp_type'] = urls.slug.apply(fetch_corp_type)

In [26]:
# urls.head()

In [27]:
# urls.to_csv("corp_type.csv")

In [28]:
# response = requests.get('https://data.medicare.gov/resource/y2hd-n93e.json')
# ownership = response.json()

In [29]:
# ownership

In [30]:
urls = pd.read_csv("corp_type.csv")

In [31]:
urls['facility_id'] = urls.URL.str.replace(r'https://www.medicare.gov/nursinghomecompare/profile.html#profTab=0&ID=', '', regex=True)

In [32]:
urls.head()

,facility_id,URL,slug,corp_type
0,015009,https://www.medicare.gov/nursinghomecompare/pr...,15009,For profit - Corporation
1,015010,https://www.medicare.gov/nursinghomecompare/pr...,15010,For profit - Corporation
2,015012,https://www.medicare.gov/nursinghomecompare/pr...,15012,Government - County
3,015014,https://www.medicare.gov/nursinghomecompare/pr...,15014,For profit - Individual
4,015015,https://www.medicare.gov/nursinghomecompare/pr...,15015,For profit - Individual


In [33]:
urls = urls.drop(['URL', 'slug'], axis=1)

In [34]:
df.shape

(360928, 13)

In [35]:
df = df.merge(urls, left_on='facility_id', right_on='facility_id')

In [36]:
df['corp_type']=df.corp_type.replace({"For profit - Corporation": "For profit", 
                           "Non profit - Corporation": "Non profit/Government", 
                           "For profit - Limited Liability company": "For profit",
                           "For profit - Partnership": "For profit",
                           "For profit - Individual": "For profit",
                           "Government - County": "Non profit/Government",
                           "Non profit - Other": "Non profit/Government",
                           "Non profit - Church related": "Non profit/Government",
                           "Government - Hospital district": "Non profit/Government",
                           "Government - State": "Non profit/Government",
                           "Government - City/county": "Non profit/Government",
                           "Government - City": "Non profit/Government",
                           "Government - Federal": "Non profit/Government"})

## List of Deficiencies and Ownership Type per Facility

In [225]:
df.head()

,facility_name,facility_id,address,city,state,zip,inspection_date,deficiency_tag,scope_severity,complaint,standard,eventid,inspection_text,corp_type
0,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,157,D,0,1,XW2G11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,For profit
1,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,166,D,1,1,XW2G11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,For profit
2,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,241,D,1,1,XW2G11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,For profit
3,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,253,E,1,1,XW2G11,"><BR/>Based on observations and interview, the...",For profit
4,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,309,D,1,1,XW2G11,**NOTE- TERMS IN BRACKETS HAVE BEEN EDITED TO ...,For profit


## Count/average of for profit and non profit homes per state

In [38]:
corp_type = df.drop_duplicates(subset='facility_name', keep='last')

In [39]:
corp_type = corp_type.pivot_table(columns=['corp_type'],
                               index=['state'],
                               aggfunc='count', fill_value=0)

In [40]:
corp_type = corp_type.drop(['city', 'complaint', 'scope_severity', 'standard', 'zip', 'inspection_text', 'inspection_date', 'facility_name', 'facility_id', 'deficiency_tag', 'eventid'], axis=1)

In [41]:
corp_type = corp_type.address.astype(str).reset_index()

In [42]:
corp_type = corp_type.apply(pd.to_numeric, errors='ignore')


In [43]:
corp_type['total_homes'] = corp_type.sum(axis = 1, skipna = True)

In [44]:
corp_type['pct_for_profit'] = (corp_type['For profit'] * 100) / corp_type['total_homes']

In [45]:
corp_type['pct_non_profit'] = (corp_type['Non profit/Government'] * 100) / corp_type['total_homes']

In [46]:
corp_type.head()

corp_type,state,For profit,Non profit/Government,Poorly formatted :(,total_homes,pct_for_profit,pct_non_profit
0,AK,1,17,0,18,5.555556,94.444444
1,AL,185,37,0,222,83.333333,16.666667
2,AR,183,44,0,227,80.616740,19.383260
3,AZ,114,27,0,141,80.851064,19.148936
4,CA,984,196,1,1181,83.319221,16.596105


## Count/average of deficiencies per state

In [47]:
deficiencies = df.pivot_table(columns='scope_severity',
                               index=['state'], aggfunc='count', fill_value=0)

In [48]:
deficiencies = deficiencies.drop(['city', 'complaint', 'standard', 'zip', 'inspection_text', 'inspection_date', 'facility_name', 'facility_id', 'deficiency_tag', 'eventid', 'corp_type'], axis=1)

In [49]:
deficiencies = deficiencies.address.astype(str).reset_index()

In [50]:
deficiencies = deficiencies.apply(pd.to_numeric, errors='ignore')


In [51]:
deficiencies['total_deficiencies'] = deficiencies.sum(axis = 1, skipna = True)

In [52]:
deficiencies.head()

scope_severity,state,B,C,D,E,F,G,H,I,J,K,L,total_deficiencies
0,AK,1,25,306,164,107,13,1,0,5,1,0,623
1,AL,12,133,1787,221,402,18,0,0,72,25,5,2675
2,AR,246,89,1068,2472,517,25,32,0,78,167,8,4702
3,AZ,38,6,1409,740,24,77,2,0,6,6,1,2309
4,CA,1577,248,29111,11725,1596,835,35,1,63,76,72,45339


In [53]:
corp_type['avg_deficiencies'] = deficiencies['total_deficiencies'] / corp_type['total_homes']

In [54]:
corp_type['total_deficiencies'] = deficiencies['total_deficiencies']

## Count/average of complaints per state

In [55]:
complaints = df.pivot_table(columns='complaint',
                               index=['state'], aggfunc='count', fill_value=0)

In [56]:
complaints = complaints.drop(['city', 'scope_severity', 'standard', 'zip', 'inspection_text', 'inspection_date', 'facility_name', 'facility_id', 'deficiency_tag', 'eventid', 'corp_type'], axis=1)

In [57]:
complaints = complaints.address.astype(str).reset_index()

In [58]:
complaints=complaints.drop([0], axis=1)

In [59]:
complaints = complaints.rename(columns={1: "complaints"})
 

In [60]:
complaints['complaints'] = complaints['complaints'].astype(int)

In [61]:
complaints.head()

complaint,state,complaints
0,AK,48
1,AL,466
2,AR,1086
3,AZ,472
4,CA,11325


In [62]:
corp_type['complaints'] = complaints['complaints']

In [63]:
corp_type['avg_complaints'] = complaints['complaints'] / corp_type['total_homes']

## Count/average of deficiencies against for profits and non profits per state

In [64]:
df2 = corp_type[['state', 'For profit', 'Non profit/Government', 'total_homes', 'pct_for_profit', 'pct_non_profit', 'total_deficiencies', 'avg_deficiencies', 'complaints', 'avg_complaints']].copy()


In [65]:
corp_deficiencies = df.pivot_table(columns='corp_type',
                               index='state', aggfunc='count', fill_value=0)

In [66]:
corp_deficiencies = corp_deficiencies.drop(['city', 'scope_severity', 'standard', 'zip', 'inspection_text', 'inspection_date', 'facility_id', 'deficiency_tag', 'complaint', 'address','eventid'], axis=1)

In [67]:
corp_deficiencies = corp_deficiencies.facility_name.astype(str)

In [68]:
corp_deficiencies=corp_deficiencies.reset_index()

In [69]:
corp_deficiencies = corp_deficiencies.apply(pd.to_numeric, errors='ignore')


In [70]:
corp_deficiencies = corp_deficiencies.rename(columns={"For profit": "for_profit_deficiencies", "Non profit/Government": "non_profit_deficiencies"})


In [71]:
corp_deficiencies['total_deficienciesFP/NP'] = corp_deficiencies['for_profit_deficiencies'] + corp_deficiencies['non_profit_deficiencies']

In [72]:
corp_deficiencies['avg_for_profit_deficiencies'] = (corp_deficiencies['for_profit_deficiencies'] * 100) / (corp_deficiencies['total_deficienciesFP/NP'])

In [73]:
corp_deficiencies['avg_non_profit_deficiencies'] = (corp_deficiencies['non_profit_deficiencies'] * 100) / (corp_deficiencies['total_deficienciesFP/NP'])

In [74]:
corp_deficiencies = corp_deficiencies.drop(['Poorly formatted :('], axis = 1)

In [75]:
df2 = df2.merge(corp_deficiencies, left_on='state', right_on='state')

In [76]:
corp_deficiencies.head()

corp_type,state,for_profit_deficiencies,non_profit_deficiencies,total_deficienciesFP/NP,avg_for_profit_deficiencies,avg_non_profit_deficiencies
0,AK,48,575,623,7.704655,92.295345
1,AL,2267,408,2675,84.747664,15.252336
2,AR,3810,892,4702,81.029349,18.970651
3,AZ,1733,576,2309,75.054136,24.945864
4,CA,40321,4995,45316,88.977403,11.022597


## Count/average of serious deficiencies against for profits and non profits per state

In [77]:
df_serious = df[(df.scope_severity == 'J') | (df.scope_severity == 'K') | (df.scope_severity == 'L')]

In [78]:
df_serious.corp_type.value_counts()

For profit               4749
Non profit/Government    1161
Poorly formatted :(        10
Name: corp_type, dtype: int64

In [79]:
df_serious = df_serious.pivot_table(columns='corp_type', index=['state'], aggfunc='count', fill_value=0)

In [80]:
df_serious = df_serious.drop(['city', 'scope_severity', 'standard', 'zip', 'inspection_text', 'inspection_date', 'facility_id', 'deficiency_tag', 'complaint', 'address','eventid'], axis=1)

In [81]:
df_serious = df_serious.facility_name.astype(str).reset_index()

In [82]:
df_serious = df_serious.rename(columns={"For profit": "for_profit_serious_deficiencies", "Non profit/Government": "non_profit_serious_deficiencies"})

In [83]:
df_serious = df_serious.drop(['Poorly formatted :('], axis=1)

In [84]:
df_serious = df_serious.apply(pd.to_numeric, errors='ignore')

In [85]:
df_serious['total_serious_deficiencies'] = df_serious['for_profit_serious_deficiencies'] + df_serious['non_profit_serious_deficiencies']

In [86]:
df_serious['avg_non_profit_serious_deficiencies'] = (df_serious['non_profit_serious_deficiencies'] * 100) / (df_serious['total_serious_deficiencies'])

In [87]:
df_serious['avg_for_profit_serious_deficiencies'] = (df_serious['for_profit_serious_deficiencies'] * 100) / (df_serious['total_serious_deficiencies'])

In [88]:
df2 = df2.merge(df_serious, left_on='state', right_on='state')

In [89]:
df_serious.head()

corp_type,state,for_profit_serious_deficiencies,non_profit_serious_deficiencies,total_serious_deficiencies,avg_non_profit_serious_deficiencies,avg_for_profit_serious_deficiencies
0,AK,0,6,6,100.000000,0.000000
1,AL,88,14,102,13.725490,86.274510
2,AR,190,63,253,24.901186,75.098814
3,AZ,8,5,13,38.461538,61.538462
4,CA,192,19,211,9.004739,90.995261


## Count/average of complaints against for profits and non profits per state

In [90]:
ownership_complaints = df.pivot_table(columns='corp_type', index=['state'], aggfunc='sum', values = 'complaint', fill_value=0).reset_index().drop(['Poorly formatted :('], axis=1)

In [91]:
ownership_complaints['For profit'] = ownership_complaints['For profit'].astype(int)

In [92]:
ownership_complaints['Non profit/Government'] = ownership_complaints['Non profit/Government'].astype(int)

In [93]:
ownership_complaints = ownership_complaints.rename(columns={'For profit': "for_profit_complaints",
                                                           'Non profit/Government': 'non_profit_complaints'})


In [94]:
ownership_complaints['total_complaintsFP/NP'] = ownership_complaints['for_profit_complaints'] + ownership_complaints['non_profit_complaints']

In [95]:
ownership_complaints['avg_for_profit_complaints'] = ownership_complaints['for_profit_complaints'] * 100 / ownership_complaints['total_complaintsFP/NP']

In [96]:
ownership_complaints['avg_non_profit_complaints'] = ownership_complaints['non_profit_complaints'] * 100 / ownership_complaints['total_complaintsFP/NP']

In [97]:
df2=df2.merge(ownership_complaints, left_on='state', right_on='state')

In [221]:
ownership_complaints['for_profit_complaints'].sum()

79824

### Conclusions

In [216]:
df.corp_type.value_counts()

For profit               274930
Non profit/Government     85641
Poorly formatted :(         357
Name: corp_type, dtype: int64

In [162]:
exampled3.to_csv('ownershipd3.csv')

In [107]:
df2.dtypes

corp_type
state                                   object
For profit                               int64
Non profit/Government                    int64
total_homes                              int64
pct_for_profit                         float64
pct_non_profit                         float64
total_deficiencies                       int64
avg_deficiencies                       float64
complaints                               int64
avg_complaints                         float64
for_profit_deficiencies                  int64
non_profit_deficiencies                  int64
total_deficienciesFP/NP                  int64
avg_for_profit_deficiencies            float64
avg_non_profit_deficiencies            float64
for_profit_serious_deficiencies          int64
non_profit_serious_deficiencies          int64
total_serious_deficiencies               int64
avg_non_profit_serious_deficiencies    float64
avg_for_profit_serious_deficiencies    float64
for_profit_complaints                    int64
non

In [108]:
df2.pct_for_profit.mean()

65.82109200653478

In [109]:
df2.pct_non_profit.mean()

34.09259838307977

In [110]:
df2.avg_non_profit_complaints.mean()

23.43036276850659

In [111]:
df2.avg_for_profit_complaints.mean()

76.56963723149342

In [112]:
df2.avg_non_profit_deficiencies.mean()

28.84679833617225

In [113]:
df2.avg_for_profit_deficiencies.mean()

71.15320166382774

In [114]:
df2.avg_non_profit_serious_deficiencies.mean()

28.027686116491928

In [115]:
df2.avg_for_profit_serious_deficiencies.mean()

71.97231388350806

In [116]:
df2[df2.state=='TX'].avg_for_profit_deficiencies.mean()

84.75945784927018

In [117]:
df2[df2.state=='TX'].avg_non_profit_deficiencies.mean()

15.240542150729818

In [118]:
staffing = pd.read_csv('PBJ_Daily_Nurse_Staffing_CY_2019Q1.csv')

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,16,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [119]:
staffing = staffing[['WorkDate', 'PROVNUM', 'PROVNAME', 'STATE', 'MDScensus', 'Hrs_RN', 'Hrs_LPN', 'Hrs_CNA', 'Hrs_NAtrn']].copy()

## Census per facility and per state during 2019 Q1 (90 days)

In [120]:
staffing = staffing.apply(pd.to_numeric, errors='ignore')

In [121]:
staffing.WorkDate.value_counts()

20190329    15058
20190121    15058
20190127    15058
20190124    15058
20190125    15058
20190130    15058
20190131    15058
20190128    15058
20190129    15058
20190202    15058
20190203    15058
20190201    15058
20190206    15058
20190207    15058
20190204    15058
20190205    15058
20190210    15058
20190211    15058
20190208    15058
20190209    15058
20190214    15058
20190126    15058
20190120    15058
20190328    15058
20190123    15058
20190103    15058
20190101    15058
20190106    15058
20190107    15058
20190104    15058
            ...  
20190321    15058
20190326    15058
20190327    15058
20190324    15058
20190325    15058
20190330    15058
20190331    15058
20190308    15058
20190311    15058
20190310    15058
20190227    15058
20190219    15058
20190216    15058
20190217    15058
20190222    15058
20190223    15058
20190220    15058
20190221    15058
20190226    15058
20190224    15058
20190305    15058
20190225    15058
20190228    15058
20190302    15058
20190303  

In [122]:
census_per_facility = staffing.groupby(by=['STATE', 'PROVNUM']).MDScensus.sum().reset_index()

In [123]:
census_per_facility['avg_MDScensus'] = census_per_facility['MDScensus'] / 90

In [124]:
census_per_facility = census_per_facility.rename(columns={'MDScensus': 'total_residents',
                                                          'avg_MDScensus': 'avg_residents'})


In [125]:
census_per_facility.head()

,STATE,PROVNUM,total_residents,avg_residents
0,AK,025010,1830,20.333333
1,AK,025018,4284,47.600000
2,AK,025019,1046,11.622222
3,AK,025020,6916,76.844444
4,AK,025021,4730,52.555556


In [126]:
census_per_state = staffing.groupby(by=['STATE']).MDScensus.sum().reset_index()

In [127]:
census_per_state['avg_residents'] = census_per_state['MDScensus'] / 90

In [128]:
census_per_state = census_per_state.rename(columns={'MDScensus': 'total_residents'})


In [129]:
census_per_state.head()

,STATE,total_residents,avg_residents
0,AK,44207,491.188889
1,AL,2025433,22504.811111
2,AR,1518636,16873.733333
3,AZ,1080635,12007.055556
4,CA,9027646,100307.177778


## Per capita

In [130]:
df2 = df2.merge(census_per_state, left_on='state', right_on='STATE')

In [131]:
df2['for_profit_deficiencies_percapita'] = df2['for_profit_deficiencies'] / df2['avg_residents']

In [132]:
df2['non_profit_deficiencies_percapita'] = df2['non_profit_deficiencies'] / df2['avg_residents']

In [133]:
df2['for_profit_seriousdeficiencies_percapita'] = df2['for_profit_serious_deficiencies'] / df2['avg_residents']

In [134]:
df2['non_profit_seriousdeficiencies_percapita'] = df2['non_profit_serious_deficiencies'] / df2['avg_residents']

In [137]:
df2['for_profit_complaints_percapita'] = df2['for_profit_complaints'] / df2['avg_residents']

In [138]:
df2['non_profit_complaints_percapita'] = df2['non_profit_complaints'] / df2['avg_residents']

In [215]:
df2[(df2.pct_for_profit) > (df2.pct_non_profit)]

,state,For profit,Non profit/Government,total_homes,pct_for_profit,pct_non_profit,total_deficiencies,avg_deficiencies,complaints,avg_complaints,...,avg_non_profit_complaints,STATE,total_residents,avg_residents,for_profit_deficiencies_percapita,non_profit_deficiencies_percapita,for_profit_seriousdeficiencies_percapita,non_profit_seriousdeficiencies_percapita,for_profit_complaints_percapita,non_profit_complaints_percapita
1,AL,185,37,222,83.333333,16.666667,2675,12.049550,466,2.099099,...,9.227468,AL,2025433,22504.811111,0.100734,0.018129,0.003910,0.000622,0.018796,0.001911
2,AR,183,44,227,80.616740,19.383260,4702,20.713656,1086,4.784141,...,21.454880,AR,1518636,16873.733333,0.225795,0.052863,0.011260,0.003734,0.050552,0.013808
3,AZ,114,27,141,80.851064,19.148936,2309,16.375887,472,3.347518,...,29.661017,AZ,1080635,12007.055556,0.144332,0.047972,0.000666,0.000416,0.027650,0.011660
4,CA,984,196,1181,83.319221,16.596105,45339,38.390347,11325,9.589331,...,7.631835,CA,9027646,100307.177778,0.401975,0.049797,0.001914,0.000189,0.104250,0.008614
5,CO,180,44,224,80.357143,19.642857,5403,24.120536,1290,5.758929,...,10.155039,CO,1479200,16435.555556,0.278968,0.049770,0.001947,0.000122,0.070518,0.007971
6,CT,171,38,209,81.818182,18.181818,4686,22.421053,1768,8.459330,...,8.710407,CT,1964335,21825.944444,0.185834,0.028865,0.001512,0.000092,0.073949,0.007056
8,DE,26,17,43,60.465116,39.534884,1645,38.255814,431,10.023256,...,27.146172,DE,358685,3985.388889,0.271743,0.141015,0.001505,0.001004,0.078788,0.029357
9,FL,477,200,677,70.457903,29.542097,13178,19.465288,2598,3.837518,...,20.900693,FL,6669198,74102.200000,0.133329,0.044506,0.002416,0.000783,0.027732,0.007328
10,GA,235,111,346,67.919075,32.080925,4009,11.586705,1206,3.485549,...,18.325041,GA,2922847,32476.077778,0.091945,0.031500,0.005450,0.001324,0.030330,0.006805
11,HI,24,20,44,54.545455,45.454545,1029,23.386364,65,1.477273,...,32.307692,HI,310843,3453.811111,0.157797,0.140135,0.002895,0.003185,0.012740,0.006080


# Complaints and deficiencies per capita vs. ownership type

### Lollipop chart

In [187]:
lollipop = df2[['state', 'for_profit_complaints_percapita', 'non_profit_complaints_percapita']]

In [189]:
lollipop['sum_percapita'] = lollipop['for_profit_complaints_percapita'] + lollipop['non_profit_complaints_percapita']

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [191]:
lollipop['avg_forprofit_complaints_percapita'] = lollipop['for_profit_complaints_percapita'] * 100 / lollipop['sum_percapita']

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [192]:
lollipop['avg_nonprofit_complaints_percapita'] = lollipop['non_profit_complaints_percapita'] * 100 / lollipop['sum_percapita']

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [194]:
lollipop = pd.melt(lollipop, id_vars=['state'], value_vars=['avg_forprofit_complaints_percapita', 'avg_nonprofit_complaints_percapita']).sort_values(by='state')

In [196]:
lollipop = lollipop.reset_index().drop(['index'], axis=1)

In [197]:
lollipop.head()

,state,variable,value
0,AK,avg_forprofit_complaints_percapita,25.000000
1,AK,avg_nonprofit_complaints_percapita,75.000000
2,AL,avg_forprofit_complaints_percapita,90.772532
3,AL,avg_nonprofit_complaints_percapita,9.227468
4,AR,avg_forprofit_complaints_percapita,78.545120


In [198]:
lollipop.to_csv('lollipop.csv')

### Pie chart multiples

In [199]:
piechart = df2[['state', 'for_profit_deficiencies_percapita', 'non_profit_deficiencies_percapita']]

In [202]:
piechart['sum_percapita'] = piechart['for_profit_deficiencies_percapita'] + piechart['non_profit_deficiencies_percapita']

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [203]:
piechart['avg_forprofit_deficiencies_percapita'] = piechart['for_profit_deficiencies_percapita'] * 100 / piechart['sum_percapita']

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [204]:
piechart['avg_nonprofit_deficiencies_percapita'] = piechart['non_profit_deficiencies_percapita'] * 100 / piechart['sum_percapita']

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [205]:
piechart = pd.melt(piechart, id_vars=['state'], value_vars=['avg_forprofit_deficiencies_percapita', 'avg_nonprofit_deficiencies_percapita']).sort_values(by='state')

In [206]:
piechart = piechart.reset_index().drop(['index'], axis=1)

In [207]:
piechart.head()

,state,variable,value
0,AK,avg_forprofit_deficiencies_percapita,7.704655
1,AK,avg_nonprofit_deficiencies_percapita,92.295345
2,AL,avg_forprofit_deficiencies_percapita,84.747664
3,AL,avg_nonprofit_deficiencies_percapita,15.252336
4,AR,avg_forprofit_deficiencies_percapita,81.029349


In [208]:
piechart.to_csv('piechart.csv')

In [140]:
df2.to_csv('ownershipdata.csv')

### RN hours per resident by state

In [131]:
staffing.head()

,WorkDate,PROVNUM,PROVNAME,STATE,MDScensus,Hrs_RN,Hrs_LPN,Hrs_CNA,Hrs_NAtrn
0,20190101,15009,"BURNS NURSING HOME, INC.",AL,52,9.5,56.00,128.09,0.0
1,20190102,15009,"BURNS NURSING HOME, INC.",AL,52,17.75,56.25,134.93,0.0
2,20190103,15009,"BURNS NURSING HOME, INC.",AL,53,24,70.00,141.35,0.0
3,20190104,15009,"BURNS NURSING HOME, INC.",AL,53,36,56.25,138.95,0.0
4,20190105,15009,"BURNS NURSING HOME, INC.",AL,51,8.5,48.10,139.95,0.0


In [133]:
staffing.Hrs_RN = staffing.Hrs_RN.replace(r',', '', regex=True)

In [134]:
staffing.Hrs_RN = staffing.Hrs_RN.astype(float)

In [143]:
rn_average = (staffing.groupby(by='STATE').Hrs_RN.sum() / 90).reset_index()

In [145]:
census_per_state.head()

,STATE,total_residents,avg_residents
0,AK,44207,491.188889
1,AL,2025433,22504.811111
2,AR,1518636,16873.733333
3,AZ,1080635,12007.055556
4,CA,9027646,100307.177778


In [146]:
rn_average = rn_average.merge(census_per_state, left_on='STATE', right_on='STATE')

In [148]:
rn_average['avg_RN'] = rn_average['Hrs_RN'] / rn_average['avg_residents']

In [174]:
rn_average.sort_values(by='avg_RN').head(20)

,STATE,Hrs_RN,total_residents,avg_residents,avg_RN
18,LA,2630.914889,2297693,25529.922222,0.103052
36,OK,2865.442333,1470539,16339.322222,0.175371
2,AR,3074.318000,1518636,16873.733333,0.182195
44,TX,18933.914889,8285514,92061.266667,0.205666
10,GA,7035.704778,2922847,32476.077778,0.216643
24,MO,9299.234000,3155864,35065.155556,0.265199
46,VA,8040.299556,2530080,28112.000000,0.286010
27,NC,10968.050111,3199837,35553.744444,0.308492
43,TN,9069.299556,2430673,27007.477778,0.335807
1,AL,7662.789778,2025433,22504.811111,0.340496


## RN hours per resident per ownership type by state

In [177]:
rn = df[['facility_id', 'corp_type']].copy()

In [179]:
rn = staffing.merge(rn, left_on='PROVNUM', right_on='facility_id')

In [180]:
rn.head()

,WorkDate,PROVNUM,PROVNAME,STATE,MDScensus,Hrs_RN,Hrs_LPN,Hrs_CNA,Hrs_NAtrn,facility_id,corp_type
0,20190105,015416,ANDALUSIA MANOR,AL,133,9.63,137.44,296.84,0.0,015416,For profit
1,20190105,015416,ANDALUSIA MANOR,AL,133,9.63,137.44,296.84,0.0,015416,For profit
2,20190105,015416,ANDALUSIA MANOR,AL,133,9.63,137.44,296.84,0.0,015416,For profit
3,20190105,015416,ANDALUSIA MANOR,AL,133,9.63,137.44,296.84,0.0,015416,For profit
4,20190105,015416,ANDALUSIA MANOR,AL,133,9.63,137.44,296.84,0.0,015416,For profit


In [181]:
rn_ownership = rn.pivot_table(columns=['corp_type'],index=['STATE'],aggfunc='sum', values='MDScensus',fill_value=0).reset_index().drop(['Poorly formatted :('], axis=1)

In [182]:
rn_ownership = rn_ownership.rename(columns={'For profit': "census_for_profit", 'Non profit/Government': 'census_non_profit'})

In [183]:
rn_ownership['avg_census_for_profit'] = (rn_ownership['census_for_profit'] / 90).round(2)

In [184]:
rn_ownership['avg_census_non_profit'] = (rn_ownership['census_non_profit'] / 90).round(2)

In [185]:
rn_ownership.head()

corp_type,STATE,census_for_profit,census_non_profit,avg_census_for_profit,avg_census_non_profit
0,AK,395808,1286499,4397.87,14294.43
1,AL,3436300,488700,38181.11,5430.00
2,AR,5375206,1646951,59724.51,18299.46
3,AZ,13781715,3675948,153130.17,40843.87
4,CA,94523736,13602296,1050263.73,151136.62


In [189]:
rn2=rn.pivot_table(columns=['corp_type'],index=['STATE'],aggfunc='sum', values='Hrs_RN',fill_value=0).reset_index()

In [190]:
rn2 = rn2.rename(columns={'For profit': "rn_for_profit", 'Non profit/Government': 'rn_non_profit'})

In [191]:
rn2['avg_rn_for_profit'] = (rn2['rn_for_profit'] / 90).round(2)

In [192]:
rn2['avg_rn_non_profit'] = (rn2['rn_non_profit'] / 90).round(2)

In [193]:
rn2.head()

corp_type,STATE,rn_for_profit,rn_non_profit,Poorly formatted :(,avg_rn_for_profit,avg_rn_non_profit
0,AK,2.901038e+05,1839143.32,0.0,3223.38,20434.93
1,AL,1.295047e+06,149389.52,0.0,14389.41,1659.88
2,AR,7.339140e+05,563619.50,0.0,8154.60,6262.44
3,AZ,5.831081e+06,1878568.11,0.0,64789.78,20872.98
4,CA,3.173649e+07,9172172.62,0.0,352627.68,101913.03


In [194]:
rn_ownership = rn_ownership.merge(rn2, left_on='STATE', right_on='STATE')

In [197]:
rn_ownership = rn_ownership.drop(['Poorly formatted :('], axis=1)

In [200]:
rn_ownership.head()

corp_type,STATE,census_for_profit,census_non_profit,avg_census_for_profit,avg_census_non_profit,rn_for_profit,rn_non_profit,avg_rn_for_profit,avg_rn_non_profit
0,AK,395808,1286499,4397.87,14294.43,2.901038e+05,1839143.32,3223.38,20434.93
1,AL,3436300,488700,38181.11,5430.00,1.295047e+06,149389.52,14389.41,1659.88
2,AR,5375206,1646951,59724.51,18299.46,7.339140e+05,563619.50,8154.60,6262.44
3,AZ,13781715,3675948,153130.17,40843.87,5.831081e+06,1878568.11,64789.78,20872.98
4,CA,94523736,13602296,1050263.73,151136.62,3.173649e+07,9172172.62,352627.68,101913.03


In [201]:
rn_ownership['avg_RN_per_resident_forprofit'] = rn_ownership['avg_rn_for_profit'] / rn_ownership['avg_census_for_profit']

In [202]:
rn_ownership['avg_RN_per_resident_nonprofit'] = rn_ownership['avg_rn_non_profit'] / rn_ownership['avg_census_non_profit']

corp_type,STATE,census_for_profit,census_non_profit,avg_census_for_profit,avg_census_non_profit,rn_for_profit,rn_non_profit,avg_rn_for_profit,avg_rn_non_profit,avg_RN_per_resident_forprofit,avg_RN_per_resident_nonprofit
0,AK,395808,1286499,4397.87,14294.43,2.901038e+05,1.839143e+06,3223.38,20434.93,0.732941,1.429573
1,AL,3436300,488700,38181.11,5430.00,1.295047e+06,1.493895e+05,14389.41,1659.88,0.376872,0.305687
2,AR,5375206,1646951,59724.51,18299.46,7.339140e+05,5.636195e+05,8154.60,6262.44,0.136537,0.342220
3,AZ,13781715,3675948,153130.17,40843.87,5.831081e+06,1.878568e+06,64789.78,20872.98,0.423103,0.511043
4,CA,94523736,13602296,1050263.73,151136.62,3.173649e+07,9.172173e+06,352627.68,101913.03,0.335752,0.674311
5,CO,6293583,1906180,69928.70,21179.78,3.480958e+06,9.051282e+05,38677.31,10056.98,0.553096,0.474839
6,CT,39421844,6436888,438020.49,71520.98,1.353287e+07,3.784944e+06,150365.26,42054.93,0.343284,0.588008
7,DC,5960386,3058610,66226.51,33984.56,3.962252e+06,2.597202e+06,44025.02,28857.80,0.664764,0.849144
8,DE,10077037,4527054,111967.08,50300.60,5.678292e+06,2.708580e+06,63092.14,30095.33,0.563488,0.598310
9,FL,12437864,5933818,138198.49,65931.31,5.476700e+06,3.188527e+06,60852.22,35428.07,0.440325,0.537348


## In-depth look at owners

In [205]:
ownership = pd.read_csv('Ownership (1).csv')

In [207]:
ownership2 = df[['facility_id', 'corp_type']].copy()

In [208]:
ownership = ownership2.merge(ownership, left_on='facility_id', right_on='Federal Provider Number')

In [209]:
ownership.head()

,facility_id,corp_type,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Role played by Owner or Manager in Facility,Owner Type,Owner Name,Ownership Percentage,Association Date,Location,Processing Date
0,075001,For profit,075001,ST JOSEPH'S CENTER,6448 MAIN STREET,TRUMBULL,CT,6611,5% OR GREATER INDIRECT OWNERSHIP INTEREST,Organization,GENESIS HOLDINGS LLC,NO PERCENTAGE PROVIDED,since 02/02/2015,"6448 MAIN STREET TRUMBULL, CT 06611 (41.27423,...",09/01/2019
1,075001,For profit,075001,ST JOSEPH'S CENTER,6448 MAIN STREET,TRUMBULL,CT,6611,OFFICER,Individual,"KIELAR, WALTER",NOT APPLICABLE,since 03/02/2015,"6448 MAIN STREET TRUMBULL, CT 06611 (41.27423,...",09/01/2019
2,075001,For profit,075001,ST JOSEPH'S CENTER,6448 MAIN STREET,TRUMBULL,CT,6611,5% OR GREATER INDIRECT OWNERSHIP INTEREST,Organization,ZAC PROPERTIES XI LLC,NO PERCENTAGE PROVIDED,since 12/01/2012,"6448 MAIN STREET TRUMBULL, CT 06611 (41.27423,...",09/01/2019
3,075001,For profit,075001,ST JOSEPH'S CENTER,6448 MAIN STREET,TRUMBULL,CT,6611,5% OR GREATER INDIRECT OWNERSHIP INTEREST,Organization,"GEN OPERATIONS I, LLC",NO PERCENTAGE PROVIDED,since 12/01/2012,"6448 MAIN STREET TRUMBULL, CT 06611 (41.27423,...",09/01/2019
4,075001,For profit,075001,ST JOSEPH'S CENTER,6448 MAIN STREET,TRUMBULL,CT,6611,OPERATIONAL/MANAGERIAL CONTROL,Individual,"DAVIS, KEITH",NOT APPLICABLE,since 12/01/2012,"6448 MAIN STREET TRUMBULL, CT 06611 (41.27423,...",09/01/2019
